# Registry Pattern: Solutions

Complete solutions to all exercises.

## Solution 1: ValidatorRegistry

In [ ]:
from typing import Callable, Tuple, Dict

class ValidatorRegistry:
    """Registry for validation functions."""
    _validators: Dict[str, Callable] = {}
    
    @classmethod
    def register(cls, func: Callable) -> Callable:
        """Register a validator function."""
        name = func.__name__
        cls._validators[name] = func
        return func
    
    @classmethod
    def get(cls, name: str) -> Callable:
        """Get validator by name."""
        if name not in cls._validators:
            available = ", ".join(cls._validators.keys()) if cls._validators else "none"
            raise ValueError(f"Validator '{name}' not registered. Available: {available}")
        return cls._validators[name]
    
    @classmethod
    def list_validators(cls) -> list[str]:
        """List all registered validators."""
        return list(cls._validators.keys())

def validator(func: Callable) -> Callable:
    """Decorator to register a validator."""
    return ValidatorRegistry.register(func)

# Test
@validator
def check_positive(value):
    if value > 0:
        return True, ""
    return False, "Value must be positive"

@validator
def check_range(value, min_val=0, max_val=100):
    if min_val <= value <= max_val:
        return True, ""
    return False, f"Value must be between {min_val} and {max_val}"

print(f"Validators: {ValidatorRegistry.list_validators()}")
validator_func = ValidatorRegistry.get('check_positive')
print(f"Valid: {validator_func(10)}")
print(f"Invalid: {validator_func(-5)}")

# Test range validator
range_validator = ValidatorRegistry.get('check_range')
print(f"\nRange test (50): {range_validator(50)}")
print(f"Range test (150): {range_validator(150)}")
print(f"Range test (50, custom): {range_validator(50, min_val=60, max_val=100)}")

## Solution 2: Parameter Validation

In [ ]:
import inspect
from typing import Any

class ValidatorRegistry:
    _validators: Dict[str, Callable] = {}
    _signatures: Dict[str, inspect.Signature] = {}
    
    @classmethod
    def register(cls, func: Callable) -> Callable:
        name = func.__name__
        cls._validators[name] = func
        cls._signatures[name] = inspect.signature(func)
        return func
    
    @classmethod
    def get(cls, name: str) -> Callable:
        if name not in cls._validators:
            raise ValueError(f"Validator '{name}' not registered")
        return cls._validators[name]
    
    @classmethod
    def validate_params(cls, name: str, params: Dict[str, Any]) -> None:
        """Validate parameters against function signature."""
        if name not in cls._signatures:
            raise ValueError(f"Validator '{name}' not registered")
        
        sig = cls._signatures[name]
        
        # Get all parameters
        func_params = sig.parameters
        
        # Check for missing required parameters
        missing = []
        for param_name, param in func_params.items():
            if param.default is inspect.Parameter.empty:
                # Required parameter
                if param_name not in params:
                    missing.append(param_name)
        
        if missing:
            raise ValueError(
                f"Missing required parameters for '{name}': {', '.join(missing)}"
            )
        
        # Check for unexpected parameters
        unexpected = set(params.keys()) - set(func_params.keys())
        if unexpected:
            raise ValueError(
                f"Unexpected parameters for '{name}': {', '.join(unexpected)}"
            )

def validator(func: Callable) -> Callable:
    return ValidatorRegistry.register(func)

# Test
@validator
def check_email(email: str, allow_subdomains: bool = True):
    if '@' in email:
        return True, ""
    return False, "Invalid email"

# Should pass
print("Test 1: Valid params")
try:
    ValidatorRegistry.validate_params('check_email', {'email': 'test@test.com'})
    print("  ✅ Valid params accepted")
except ValueError as e:
    print(f"  ❌ {e}")

# Should fail - missing required
print("\nTest 2: Missing required param")
try:
    ValidatorRegistry.validate_params('check_email', {})
    print("  ❌ Should have failed")
except ValueError as e:
    print(f"  ✅ Caught: {e}")

# Should fail - unexpected param
print("\nTest 3: Unexpected param")
try:
    ValidatorRegistry.validate_params('check_email', {'email': 'test', 'invalid': 'param'})
    print("  ❌ Should have failed")
except ValueError as e:
    print(f"  ✅ Caught: {e}")

# Should pass - with optional param
print("\nTest 4: With optional param")
try:
    ValidatorRegistry.validate_params('check_email', {'email': 'test@test.com', 'allow_subdomains': False})
    print("  ✅ Valid params with optional accepted")
except ValueError as e:
    print(f"  ❌ {e}")

## Solution 3: Plugin System with Metadata

In [ ]:
from typing import Optional, Dict, Any, Callable

class PluginRegistry:
    def __init__(self):
        self._plugins: Dict[str, Callable] = {}
        self._metadata: Dict[str, Dict[str, Any]] = {}
    
    def register(
        self,
        name: str,
        func: Callable,
        version: str,
        author: str,
        dependencies: Optional[list] = None
    ) -> Callable:
        """Register a plugin with metadata."""
        # Check for duplicates
        if name in self._plugins:
            raise ValueError(f"Plugin '{name}' already registered")
        
        # Store function
        self._plugins[name] = func
        
        # Store metadata
        self._metadata[name] = {
            'version': version,
            'author': author,
            'dependencies': dependencies or [],
            'docstring': func.__doc__
        }
        
        return func
    
    def get(self, name: str) -> Optional[Callable]:
        """Get plugin by name."""
        return self._plugins.get(name)
    
    def get_metadata(self, name: str) -> Optional[Dict[str, Any]]:
        """Get plugin metadata."""
        return self._metadata.get(name)
    
    def list_all(self) -> Dict[str, Dict[str, Any]]:
        """List all plugins with metadata."""
        return {
            name: {'func': func, **self._metadata[name]}
            for name, func in self._plugins.items()
        }

# Global registry
_registry = PluginRegistry()

def plugin(name: str, version: str, author: str, dependencies: Optional[list] = None):
    """Decorator factory for plugins."""
    def decorator(func: Callable) -> Callable:
        _registry.register(
            name=name,
            func=func,
            version=version,
            author=author,
            dependencies=dependencies
        )
        return func
    return decorator

# Test
@plugin("csv_exporter", version="1.0.0", author="Alice", dependencies=["pandas"])
def export_csv(data, filename):
    '''Export data to CSV file.'''
    return f"Exported to {filename}"

@plugin("json_parser", version="2.1.0", author="Bob")
def parse_json(data):
    '''Parse JSON data into dictionary.'''
    return {"parsed": True}

@plugin("xml_transformer", version="1.5.2", author="Charlie", dependencies=["lxml", "xmltodict"])
def transform_xml(xml_string):
    '''Transform XML string to structured format.'''
    return {"transformed": True}

# Test listing
all_plugins = _registry.list_all()
print("Registered Plugins:")
print("=" * 60)
for name, details in all_plugins.items():
    print(f"\n{name} v{details['version']}")
    print(f"  Author: {details['author']}")
    print(f"  Dependencies: {', '.join(details['dependencies']) if details['dependencies'] else 'None'}")
    print(f"  Doc: {details['docstring'].strip()}")

# Test retrieval
print("\n" + "=" * 60)
print("\nTesting plugin execution:")
csv_plugin = _registry.get('csv_exporter')
print(f"CSV Export: {csv_plugin({'name': 'John'}, 'output.csv')}")

json_plugin = _registry.get('json_parser')
print(f"JSON Parse: {json_plugin('{\"name\": \"John\"}')}")

## Solution 4: Version Tracking

In [ ]:
from typing import Optional, List, Dict, Any, Callable

class VersionedPluginRegistry:
    def __init__(self):
        # {name: {version: func}}
        self._plugins: Dict[str, Dict[str, Callable]] = {}
        # {name: {version: metadata}}
        self._metadata: Dict[str, Dict[str, Dict[str, Any]]] = {}
    
    def register(self, name: str, func: Callable, version: str, **metadata) -> Callable:
        """Register a plugin version."""
        # Initialize plugin name if first version
        if name not in self._plugins:
            self._plugins[name] = {}
            self._metadata[name] = {}
        
        # Check if version already exists
        if version in self._plugins[name]:
            raise ValueError(f"Plugin '{name}' version '{version}' already registered")
        
        # Store function and metadata
        self._plugins[name][version] = func
        self._metadata[name][version] = metadata
        
        return func
    
    def get(self, name: str, version: Optional[str] = None) -> Optional[Callable]:
        """Get plugin, optionally by version (defaults to latest)."""
        if name not in self._plugins:
            return None
        
        # If version specified, return that version
        if version is not None:
            return self._plugins[name].get(version)
        
        # Otherwise, return latest version
        versions = self.get_versions(name)
        if not versions:
            return None
        
        latest_version = versions[-1]  # Last in sorted list
        return self._plugins[name][latest_version]
    
    def get_versions(self, name: str) -> List[str]:
        """Get all versions of a plugin, sorted."""
        if name not in self._plugins:
            return []
        
        versions = list(self._plugins[name].keys())
        # Sort by parsed version tuple
        versions.sort(key=self._parse_version)
        return versions
    
    def _parse_version(self, version: str) -> tuple:
        """Parse semantic version string to tuple for comparison."""
        try:
            return tuple(int(x) for x in version.split('.'))
        except ValueError:
            # If parsing fails, return as-is for string comparison
            return (version,)
    
    def list_all(self) -> Dict[str, List[str]]:
        """List all plugins with their versions."""
        return {
            name: self.get_versions(name)
            for name in self._plugins.keys()
        }

# Test
registry = VersionedPluginRegistry()

def export_csv_v1(data, filename):
    '''Export CSV version 1 - basic export.'''
    return f"v1: Exported to {filename}"

def export_csv_v2(data, filename, encoding='utf-8'):
    '''Export CSV version 2 - with encoding support.'''
    return f"v2: Exported to {filename} ({encoding})"

def export_csv_v3(data, filename, encoding='utf-8', compression=None):
    '''Export CSV version 3 - with encoding and compression.'''
    comp_str = f", compressed={compression}" if compression else ""
    return f"v3: Exported to {filename} ({encoding}{comp_str})"

# Register multiple versions
registry.register('csv_exporter', export_csv_v1, '1.0.0', author='Alice')
registry.register('csv_exporter', export_csv_v2, '2.0.0', author='Alice')
registry.register('csv_exporter', export_csv_v3, '2.1.0', author='Bob')

# Also register another plugin
def parse_json_v1(data):
    '''Parse JSON v1.'''
    return "v1: parsed"

registry.register('json_parser', parse_json_v1, '1.0.0', author='Charlie')

print("All plugins:")
for name, versions in registry.list_all().items():
    print(f"  {name}: {', '.join(versions)}")

print("\nVersion tests for csv_exporter:")
print(f"  Versions: {registry.get_versions('csv_exporter')}")
print(f"  Latest: {registry.get('csv_exporter')({'data': 1}, 'test.csv')}")
print(f"  v1.0.0: {registry.get('csv_exporter', '1.0.0')({'data': 1}, 'test.csv')}")
print(f"  v2.0.0: {registry.get('csv_exporter', '2.0.0')({'data': 1}, 'test.csv', encoding='latin1')}")
print(f"  v2.1.0: {registry.get('csv_exporter', '2.1.0')({'data': 1}, 'test.csv', compression='gzip')}")

## Solution 5: Tag-Based Discovery

In [ ]:
from typing import Optional, List, Dict, Callable

class TaggedPluginRegistry:
    def __init__(self):
        self._plugins: Dict[str, Callable] = {}
        self._tags: Dict[str, list] = {}  # plugin_name -> [tags]
    
    def register(self, name: str, func: Callable, tags: Optional[list] = None) -> Callable:
        """Register plugin with tags."""
        if name in self._plugins:
            raise ValueError(f"Plugin '{name}' already registered")
        
        self._plugins[name] = func
        self._tags[name] = tags or []
        return func
    
    def find_by_tag(self, tag: str) -> List[str]:
        """Find all plugins with given tag."""
        result = []
        for plugin_name, plugin_tags in self._tags.items():
            if tag in plugin_tags:
                result.append(plugin_name)
        return result
    
    def find_by_tags(self, tags: List[str], match_all: bool = True) -> List[str]:
        """Find plugins by multiple tags.
        
        Args:
            tags: Tags to search for
            match_all: If True, plugin must have ALL tags (AND logic)
                       If False, plugin must have ANY tag (OR logic)
        """
        result = []
        
        for plugin_name, plugin_tags in self._tags.items():
            plugin_tag_set = set(plugin_tags)
            search_tag_set = set(tags)
            
            if match_all:
                # AND logic - plugin must have all search tags
                if search_tag_set.issubset(plugin_tag_set):
                    result.append(plugin_name)
            else:
                # OR logic - plugin must have at least one search tag
                if plugin_tag_set.intersection(search_tag_set):
                    result.append(plugin_name)
        
        return result

# Test
registry = TaggedPluginRegistry()

def export_csv(data): '''Export CSV.'''; return "csv exported"
def export_json(data): '''Export JSON.'''; return "json exported"
def export_xml(data): '''Export XML.'''; return "xml exported"
def import_csv(file): '''Import CSV.'''; return "csv imported"
def import_json(file): '''Import JSON.'''; return "json imported"
def transform_data(data): '''Transform data.'''; return "transformed"

registry.register('csv_export', export_csv, tags=['export', 'csv', 'io', 'data'])
registry.register('json_export', export_json, tags=['export', 'json', 'io', 'data'])
registry.register('xml_export', export_xml, tags=['export', 'xml', 'io'])
registry.register('csv_import', import_csv, tags=['import', 'csv', 'io', 'data'])
registry.register('json_import', import_json, tags=['import', 'json', 'io', 'data'])
registry.register('data_transform', transform_data, tags=['transform', 'data'])

print("Tag Discovery Tests:")
print("=" * 60)

print(f"\nExport plugins: {registry.find_by_tag('export')}")
print(f"CSV plugins: {registry.find_by_tag('csv')}")
print(f"IO plugins: {registry.find_by_tag('io')}")
print(f"Data plugins: {registry.find_by_tag('data')}")

print("\nMulti-tag searches (AND logic):")
print(f"Export + JSON: {registry.find_by_tags(['export', 'json'], match_all=True)}")
print(f"CSV + Export: {registry.find_by_tags(['csv', 'export'], match_all=True)}")
print(f"IO + Data: {registry.find_by_tags(['io', 'data'], match_all=True)}")

print("\nMulti-tag searches (OR logic):")
print(f"Export OR Import: {registry.find_by_tags(['export', 'import'], match_all=False)}")
print(f"CSV OR JSON: {registry.find_by_tags(['csv', 'json'], match_all=False)}")
print(f"Transform OR Export: {registry.find_by_tags(['transform', 'export'], match_all=False)}")

## Solution 6: Complete Test Suite

In [ ]:
from typing import Dict, Any, Optional, Callable

class TransformationRegistry:
    def __init__(self):
        self._transformations: Dict[str, Callable] = {}
        self._metadata: Dict[str, Dict[str, Any]] = {}
    
    def register(self, name: str, func: Callable, version: str = "1.0.0",
                 category: Optional[str] = None, tags: Optional[list] = None) -> Callable:
        if name in self._transformations:
            raise ValueError(f"Transformation '{name}' already registered")
        if not func.__doc__ or len(func.__doc__.strip()) < 10:
            raise ValueError(f"Transformation must have docstring (min 10 chars)")
        self._transformations[name] = func
        self._metadata[name] = {
            'version': version, 'category': category,
            'tags': tags or [], 'docstring': func.__doc__
        }
        return func
    
    def get(self, name: str) -> Optional[Callable]:
        return self._transformations.get(name)
    
    def get_metadata(self, name: str) -> Optional[Dict[str, Any]]:
        return self._metadata.get(name)
    
    def list_all(self) -> Dict[str, Dict[str, Any]]:
        return {
            name: {'func': func, **self._metadata[name]}
            for name, func in self._transformations.items()
        }
    
    def unregister(self, name: str) -> bool:
        if name in self._transformations:
            del self._transformations[name]
            del self._metadata[name]
            return True
        return False
    
    def clear(self):
        self._transformations.clear()
        self._metadata.clear()

# Test Functions
def test_registration():
    """Test that valid registration works."""
    registry = TransformationRegistry()
    
    def sample_transform(df):
        """Sample transformation for testing purposes."""
        return df
    
    registry.register('sample', sample_transform, version='1.0.0')
    assert 'sample' in registry.list_all(), "Registration failed"
    assert registry.get('sample') is sample_transform, "Function not stored correctly"

def test_duplicate_rejection():
    """Test that duplicate names are rejected."""
    registry = TransformationRegistry()
    
    def sample_transform(df):
        """Sample transformation for testing."""
        return df
    
    registry.register('sample', sample_transform)
    
    try:
        registry.register('sample', sample_transform)
        assert False, "Should have rejected duplicate"
    except ValueError as e:
        assert "already registered" in str(e)

def test_docstring_validation():
    """Test that functions without docstrings are rejected."""
    registry = TransformationRegistry()
    
    # No docstring
    def no_doc(df): return df
    
    try:
        registry.register('nodoc', no_doc)
        assert False, "Should have rejected missing docstring"
    except ValueError as e:
        assert "docstring" in str(e).lower()
    
    # Short docstring
    def short_doc(df): """Hi"""; return df
    
    try:
        registry.register('shortdoc', short_doc)
        assert False, "Should have rejected short docstring"
    except ValueError as e:
        assert "docstring" in str(e).lower()

def test_metadata_retrieval():
    """Test metadata retrieval."""
    registry = TransformationRegistry()
    
    def sample_transform(df):
        """Sample transformation for testing."""
        return df
    
    registry.register(
        'sample', 
        sample_transform, 
        version='2.1.0', 
        category='testing',
        tags=['test', 'sample']
    )
    
    metadata = registry.get_metadata('sample')
    assert metadata is not None, "Metadata not found"
    assert metadata['version'] == '2.1.0', "Version incorrect"
    assert metadata['category'] == 'testing', "Category incorrect"
    assert 'test' in metadata['tags'], "Tags incorrect"

def test_unregister():
    """Test unregister removes function."""
    registry = TransformationRegistry()
    
    def sample_transform(df):
        """Sample transformation."""
        return df
    
    registry.register('sample', sample_transform)
    assert 'sample' in registry.list_all(), "Registration failed"
    
    result = registry.unregister('sample')
    assert result == True, "Unregister should return True"
    assert 'sample' not in registry.list_all(), "Unregister incomplete"
    
    # Unregister non-existent
    result = registry.unregister('nonexistent')
    assert result == False, "Unregister non-existent should return False"

def test_clear():
    """Test clear removes all functions."""
    registry = TransformationRegistry()
    
    def transform1(df): """Transform 1."""; return df
    def transform2(df): """Transform 2."""; return df
    
    registry.register('t1', transform1)
    registry.register('t2', transform2)
    
    assert len(registry.list_all()) == 2, "Should have 2 registrations"
    
    registry.clear()
    assert len(registry.list_all()) == 0, "Clear failed"
    assert registry.get('t1') is None, "Functions not cleared"
    assert registry.get_metadata('t1') is None, "Metadata not cleared"

def test_list_all():
    """Test list_all returns all registered functions."""
    registry = TransformationRegistry()
    
    def transform1(df): """Transform 1."""; return df
    def transform2(df): """Transform 2."""; return df
    
    registry.register('t1', transform1, version='1.0.0', category='cat1')
    registry.register('t2', transform2, version='2.0.0', category='cat2')
    
    all_transforms = registry.list_all()
    assert len(all_transforms) == 2, "Should list 2 transforms"
    assert 't1' in all_transforms, "t1 missing"
    assert 't2' in all_transforms, "t2 missing"
    assert all_transforms['t1']['version'] == '1.0.0', "Metadata incorrect"
    assert all_transforms['t2']['category'] == 'cat2', "Metadata incorrect"

# Run all tests
def run_tests():
    tests = [
        test_registration,
        test_duplicate_rejection,
        test_docstring_validation,
        test_metadata_retrieval,
        test_unregister,
        test_clear,
        test_list_all
    ]
    
    passed = 0
    failed = 0
    
    for test in tests:
        try:
            test()
            print(f"✅ {test.__name__}")
            passed += 1
        except AssertionError as e:
            print(f"❌ {test.__name__}: {e}")
            failed += 1
        except Exception as e:
            print(f"⚠️  {test.__name__}: {e}")
            failed += 1
    
    print(f"\n{'='*60}")
    print(f"Results: {passed} passed, {failed} failed")
    if failed == 0:
        print("🎉 All tests passed!")

run_tests()

## Bonus Solution: Category-Based Registry

In [ ]:
from typing import Dict, List, Callable, Optional
from collections import defaultdict

class CategoryRegistry:
    """Registry that organizes plugins by category."""
    
    def __init__(self):
        # {category: {name: func}}
        self._categories: Dict[str, Dict[str, Callable]] = defaultdict(dict)
        # {name: [categories]} - for reverse lookup
        self._plugin_categories: Dict[str, List[str]] = {}
    
    def register(self, name: str, func: Callable, categories: List[str]) -> Callable:
        """Register plugin in one or more categories."""
        if not categories:
            raise ValueError("Plugin must have at least one category")
        
        # Store in each category
        for category in categories:
            if name in self._categories[category]:
                raise ValueError(f"Plugin '{name}' already in category '{category}'")
            self._categories[category][name] = func
        
        # Track categories for this plugin
        self._plugin_categories[name] = categories
        
        return func
    
    def get_category(self, category: str) -> Dict[str, Callable]:
        """Get all plugins in a category."""
        return dict(self._categories.get(category, {}))
    
    def list_categories(self) -> List[str]:
        """Get all categories."""
        return sorted(self._categories.keys())
    
    def get_plugin_categories(self, name: str) -> List[str]:
        """Get categories for a specific plugin."""
        return self._plugin_categories.get(name, [])
    
    def get(self, name: str) -> Optional[Callable]:
        """Get plugin by name (searches all categories)."""
        for category_plugins in self._categories.values():
            if name in category_plugins:
                return category_plugins[name]
        return None

# Test
registry = CategoryRegistry()

def export_csv(data): '''Export CSV.'''; return "csv export"
def export_json(data): '''Export JSON.'''; return "json export"
def import_csv(file): '''Import CSV.'''; return "csv import"
def transform_csv(data): '''Transform CSV.'''; return "csv transform"
def validate_json(data): '''Validate JSON.'''; return "json validate"

# Register plugins (some in multiple categories)
registry.register('csv_export', export_csv, categories=['export', 'csv', 'io'])
registry.register('json_export', export_json, categories=['export', 'json', 'io'])
registry.register('csv_import', import_csv, categories=['import', 'csv', 'io'])
registry.register('csv_transform', transform_csv, categories=['transform', 'csv'])
registry.register('json_validate', validate_json, categories=['validation', 'json'])

print("Category-Based Registry:")
print("=" * 60)

print(f"\nAll categories: {registry.list_categories()}")

print("\nPlugins by category:")
for category in registry.list_categories():
    plugins = registry.get_category(category)
    print(f"  {category}: {list(plugins.keys())}")

print("\nPlugin category lookup:")
for plugin_name in ['csv_export', 'csv_transform', 'json_validate']:
    categories = registry.get_plugin_categories(plugin_name)
    print(f"  {plugin_name}: {categories}")

print("\nExport plugins:")
export_plugins = registry.get_category('export')
for name, func in export_plugins.items():
    print(f"  {name}: {func(None)}")

print("\nCSV plugins (all categories):")
csv_plugins = registry.get_category('csv')
for name in csv_plugins.keys():
    print(f"  {name}")